# Classification non supervisée : DBSCAN

Dans ce TP, nous allons apprendra à utilier notre premier algorithme de clustering : DBSCAN. Avant cela, il est nécessaire de faire une veille sur ce sujet. Pour cela, vous devez faire des recherches sur: 
- La classification non supervisée appelée aussi clustering
- L'algorithme DBSCAN. Ce lien vous sera utile : https://openclassrooms.com/fr/courses/4379436-explorez-vos-donnees-avec-des-algorithmes-non-supervises/4379571-partitionnez-vos-donnees-avec-dbscan

Les algorithmes de clustering font partie de la classe des algorithmes d’apprentissage non supervisés. Il existe différents algorithmes de clustering parmis lesquels on peut citer : la classification ascendante hierarchique, K-means, et DBSCAN. Nous aborderons les autres algorithmes en projet.

### Clustering

Le **clustering** vise à déterminer une segmentation de la population étudiée sans a priori sur le nombre de classes, et à interpréter à posteriori les groupes ainsi créés. Ici, l’homme n’assiste pas la machine dans sa découverte des différentes typologies puisqu’aucune variable cible n’est fournie à l’algorithme durant sa phase d’apprentissage.

Là encore, les exemples d’application sont divers et variés. Dans le monde de l’entreprise, on rencontre ce sous-domaine du machine learning à travers la segmentation de clientèle, sujet d’une importance considérable dans le milieu marketing. Un autre cas d’usage correspond à la détection d’outliers, applicable à de nombreux domaines. On pourra penser plus particulièrement à la détection de fraudes, que ce soit dans les transports en commun, dans le cadre d’un remboursement de santé complémentaire ou encore au sujet de la consommation électrique… Les applications sont nombreuses.

#### Distances et similarités

Faire un clustering, c'est regrouper ensemble les points les plus proches, ou les plus semblables. Le concept de clustering repose fortement sur ceux de distance et de similarité.

Ces concepts vont nous être très utiles pour formaliser à quel point deux observations sont proches les unes des autres ; à quel point une observation est proche d'un cluster ; et à quel point deux clusters sont proches les uns des autres.

Les exemples les plus utilisés de distances sont

la distance euclidienne : 

$$ d(u,v) =  \lVert u-v \rVert_{2} =\sqrt{    \sum_{i=1}^{p} (u_i-v_i)^2  }$$

la distance de Manhattan : 

$$ d(u,v) =  \lVert u-v \rVert_{1} =\sum_{i=1}^{p}  |u_i-v_i| $$

ainsi appelée parce qu'elle correspond en deux dimension à la distance parcourue par un taxi dans les rues de Manhattan, qui sont toutes soit parallèles soient perpendiculaires les unes aux autres.

On peut utiliser une distance pour une définir une similarité : plus deux points sont distants, moins ils sont similaires, et inversement. On peut par exemple transformer une distance d en similarité s de la façon suivante :  

$$ s(u,v)=\frac{1}{1+d(u,v)} $$ 

Une autre façon courante de définir une similarité est d'utiliser la corrélation de Pearson entre les variables :


$$ \rho(u,v) = \frac{\sum_{i=1}^{p} (u_i-\overline{u})(v_i-\overline{v})}{\sqrt{    \sum_{i=1}^{p}(u_i-\overline{u}) }\sqrt{ \sum_{i=1}^{p} (v_i-\overline{v})   }} $$ 


#### Forme des clusters
Nous voulons de nos clusters qu'ils soient
- resserrés sur eux mêmes : deux points qui sont proches doivent appartenir au même cluster
- loin les uns des autres : deux points qui sont éloignés doivent appartenir à des clusters différents.

Notons d la distance que nous allons utiliser, par exemple la distance euclidienne. On appelle centroïde d'un cluster le barycentre des points de ce cluster : 
$$ μ_k = \frac{1}{|C_k|}\sum_{x∈C_k}x $$.

<div>
<img src="centroid.png" width="300"/>
</div>

On définit ainsi l'homogénéité (notée $T_k$ pour tightness en anglais) d'un cluster comme la moyenne des distances de chacun des points contenus dans ce cluster au centroïde $μ_k$:

$$ T_k = \frac{1}{|C_k|}\sum_{x∈C_k}d(x,μ_k)$$ 

Un cluster resserré aura une hétérogénéité plus faible qu'un cluster de points épars.

Pour caractériser non pas un cluster, mais l'ensemble des clusters, on peut calculer la moyenne des homogénéités de chaque cluster :

$$ T = \frac{1}{k} \sum_{k=1}^{K} T_k$$ 

Nous avions dit plus haut aussi vouloir que les clusters soient loin les uns des autres. Pour quantifier cela, on définit la séparation de deux clusters comme la distance entre leurs centroïdes : $ S_{k,l}=d(μ_k,μ_l)$. Encore une fois, on peut calculer la moyenne de ces quantités sur l'ensemble des paires de cluster (k,l) obtenues :

$$ S = \frac{2}{K(K-1)}\sum_{k=1}^{K}\sum_{l=k+1}^{K}S_{k,l} $$ 

Nous avons maintenant deux critères à optimiser. Pour nous faciliter la tâche, nous pouvons les regrouper en un seul critère, l'indice de Davies-Bouldin. L'idée de cette indice est de comparer les distannce intra-cluster (c'est l'homogénéité), que l'on veut faibles, aux distances inter-cluster (la séparation), que l'on veut grandes. Pour un cluster k, cet indice est donné par  Dk=maxl≠kTl+TkSk,l et est d'autant plus faible que tous les clusters sont homogènes (le numérateur de cette fraction est petit) et que tous sont bien séparés (le dénominateur est grand). Encore une fois, on peut calculer un indice de Davies-Bouldin global en moyennant les indices de Davies-Bouldin de tous les clusters :

$$ D = \frac{1}{K} \sum_{k=1}^{K}D_k $$ 

Une autre façon de quantifier à quel point un clustering répond à ces deux exigences (homogénéité et séparation) est de mesurer ce que l'on appelle le coefficient de silhouette. Pour un point x donné, le coefficient de silhouette s(x) permet d'évaluer si ce point appartient au « bon » cluster : est-il proche des points du cluster auquel il appartient ? Est-il loin des autres points ? Pour répondre à la première question, on calcule la distance moyenne de x à tous les autres points du cluster C_k auquel il appartient : 

$$ a(x) = \frac{1}{|Ck|−1}\sum_{u∈C_k;u\neq x}d(u,x) $$

Pour répondre à la deuxième, on calcule la plus petite valeur que pourrait prendre a(x), si x était assigné à un autre cluster :

$$ b(x) = min_{l≠k}\frac{1}{|Cl|} \sum_{u∈C_l}d(u,x) $$

Si x a été correctement assigné, alors a(x) < b(x). Le coefficient de silhouette est donné par 

$$ s(x) = \frac{ b(x)−a(x) } {max(a(x),b(x))} $$  

Il est donc compris entre -1 et 1, et d'autant plus proche de 1 que l'assignation de x à son cluster est satisfaisante. Pour évaluer un clustering, on peut calculer son coefficient de silhouette moyen.

### DBScan algorithm 



**Density Based Spatial Clustering of Applications with Noise**



<div>
<img src="unnamed.png" width="300"/>
</div>

**1. Charger les données X_train et X_test depuis les CSV.**

In [1]:
#Analyse exploratoire des données et Preprocessing
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.cluster import DBSCAN
sns.set(style="white") #white background style for seaborn plots
sns.set(style="whitegrid", color_codes=True)

In [2]:
data_image_X_train=pd.read_csv('X_train.csv', sep=',', encoding='utf8')
data_image_X_train.head(10)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data_image_X_test=pd.read_csv('X_test.csv', sep=',', encoding='utf8')
data_image_X_test.head(10)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**2. A l'aide de la fonction concat, regroupez vos dataframes.**

In [4]:
print(data_image_X_test.shape)
print(data_image_X_train.shape)

(10000, 785)
(60000, 785)


In [5]:
data = pd.concat([data_image_X_train,data_image_X_test], axis=0)

In [6]:
data.head(10)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data.shape

(70000, 785)

### 3. Transformer vos données en float et diviser par 255 comme précédemment.

In [8]:
data=data/255

In [9]:
data.head(10)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.003922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.007843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.011765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.015686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.019608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.023529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.027451,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.031373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.035294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**4. Lancer votre modèle DBScann. Obtenez-vous les 10 classes attentdues (Chiffres de 0 à 9)**

In [10]:
data.sample(frac=0.2, replace=True, random_state=1)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
5192,20.360784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50057,196.301961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21440,84.078431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20609,80.819608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49100,192.549020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22530,88.352941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25078,98.345098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21972,86.164706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22302,87.458824,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
data2 = data.sample(frac=0.2, replace=True, random_state=1)
print(data2.shape)

(14000, 785)


In [12]:
from sklearn.cluster import DBSCAN
import numpy as np
clustering = DBSCAN(eps=0.3, min_samples=5).fit(data2)
clustering.labels_
print(clustering)

DBSCAN(algorithm='auto', eps=0.3, leaf_size=30, metric='euclidean',
       metric_params=None, min_samples=5, n_jobs=None, p=None)


In [13]:
#etapes1 définir les parametre 
#jeu de donner
#transformer les donner en array en  minimun l'échantillon  
list_samples = [500, 1000, 3000, 4000, 5000]

In [14]:
from joblib import parallel_backend
from sklearn.cluster import DBSCAN
with parallel_backend('threading', n_jobs=2):
    clustering = DBSCAN(eps=3, min_samples=2).fit(data2)
    print(clustering.core_sample_indices_,'\n',clustering.components_,'\n',clustering.labels_)

[    2    15    17 ... 13995 13997 13999] 
 [[ 84.07843137   0.           0.         ...   0.           0.
    0.        ]
 [ 78.21960784   0.           0.         ...   0.           0.
    0.        ]
 [125.32941176   0.           0.         ...   0.           0.
    0.        ]
 ...
 [ 88.35294118   0.           0.         ...   0.           0.
    0.        ]
 [ 86.16470588   0.           0.         ...   0.           0.
    0.        ]
 [ 71.62745098   0.           0.         ...   0.           0.
    0.        ]] 
 [  -1   -1    0 ... 1288   -1 1351]


In [15]:
list_samples = [500, 1000, 3000, 4000, 5000]

In [16]:

with parallel_backend('threading', n_jobs=2):
    clustering = DBSCAN(eps=3, min_samples=2).fit(data2)
    print(clustering.core_sample_indices_,'\n',clustering.components_,'\n',clustering.labels_)

[    2    15    17 ... 13995 13997 13999] 
 [[ 84.07843137   0.           0.         ...   0.           0.
    0.        ]
 [ 78.21960784   0.           0.         ...   0.           0.
    0.        ]
 [125.32941176   0.           0.         ...   0.           0.
    0.        ]
 ...
 [ 88.35294118   0.           0.         ...   0.           0.
    0.        ]
 [ 86.16470588   0.           0.         ...   0.           0.
    0.        ]
 [ 71.62745098   0.           0.         ...   0.           0.
    0.        ]] 
 [  -1   -1    0 ... 1288   -1 1351]


In [17]:
with parallel_backend('threading', n_jobs=2):
    clustering = DBSCAN(eps=3, min_samples=2).fit(data2)
    print(clustering.core_sample_indices_,'\n',clustering.components_,'\n',clustering.labels_)

[    2    15    17 ... 13995 13997 13999] 
 [[ 84.07843137   0.           0.         ...   0.           0.
    0.        ]
 [ 78.21960784   0.           0.         ...   0.           0.
    0.        ]
 [125.32941176   0.           0.         ...   0.           0.
    0.        ]
 ...
 [ 88.35294118   0.           0.         ...   0.           0.
    0.        ]
 [ 86.16470588   0.           0.         ...   0.           0.
    0.        ]
 [ 71.62745098   0.           0.         ...   0.           0.
    0.        ]] 
 [  -1   -1    0 ... 1288   -1 1351]


In [18]:
with parallel_backend('threading', n_jobs=2):
    clustering = DBSCAN(eps=20, min_samples=10).fit(data2)
    print(clustering.core_sample_indices_,'\n',clustering.components_,'\n',clustering.labels_)

[    0     1     2 ... 13997 13998 13999] 
 [[ 20.36078431   0.           0.         ...   0.           0.
    0.        ]
 [196.30196078   0.           0.         ...   0.           0.
    0.        ]
 [ 84.07843137   0.           0.         ...   0.           0.
    0.        ]
 ...
 [ 86.16470588   0.           0.         ...   0.           0.
    0.        ]
 [ 87.45882353   0.           0.         ...   0.           0.
    0.        ]
 [ 71.62745098   0.           0.         ...   0.           0.
    0.        ]] 
 [0 0 0 ... 0 0 0]


In [30]:
from sklearn.datasets import load_digits
digits = load_digits()
data = digits.data
n_samples, n_features = data.shape 
n_digits = len(np.unique(digits.target))
labels = digits.target
print("n_digits: %d, \t n_samples %d, \t n_features %d" % (n_digits, n_samples, n_features))

n_digits: 10, 	 n_samples 1797, 	 n_features 64


In [44]:
db = DBSCAN(eps=28,min_samples=15)
db.fit(digits.data)
db.labels_
#print(db.labels_)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)